# PHYS 3009: TeV Gamma-Ray Data Analysis with GammaPy

## Import modules
We will see later what they are good for.

In [ ]:
%matplotlib inline
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
import matplotlib.pyplot as plt
import numpy as np
from regions import CircleSkyRegion
import scipy.stats

import os
import requests
import tarfile

In [ ]:
from gammapy.analysis import Analysis, AnalysisConfig
from gammapy.makers import  (
    SafeMaskMaker,
    SpectrumDatasetMaker,
    ReflectedRegionsBackgroundMaker,
    RingBackgroundMaker,
)
from gammapy.estimators import (
    ExcessMapEstimator,
    FluxPointsEstimator,
)
from gammapy.maps import Map, WcsNDMap
from gammapy.datasets import MapDatasetOnOff
from gammapy.data import EventList
from regions import CircleSkyRegion
from gammapy.modeling import Fit
from gammapy.data import DataStore
from gammapy.datasets import (
    Datasets,
    SpectrumDataset,
    SpectrumDatasetOnOff,
    FluxPointsDataset,
)
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    ExpCutoffPowerLawSpectralModel,
    create_crab_spectral_model,
    SkyModel,
)
from gammapy.visualization import plot_spectrum_datasets_off_regions

## Load the data
The following line defines where you have or where you want to install the data on your computer.

In [ ]:
#path = os.path.expandvars("hess_dl3_dr1")
path = 'gammapy-tutorials/datasets/hess-dl3-dr1'

The next cell will download the gammapy tutorials including the HESS data set (total size 180MB) if it is not there already.

In [ ]:
if not os.path.exists(os.path.join(path, 'hdu-index.fits.gz')):
    os.system('gammapy download tutorials --release 0.17')

Let's check again if the download is complete.

In [ ]:
if not os.path.exists(os.path.join(path, 'hdu-index.fits.gz')):
    raise Exception("gammapy-data repository not found!")
else:
    print("Great your setup is correct!")

## Preparation
We set the source position. frame='icrs' indicates that we are using coordinates in right ascension and declination.

In [ ]:
source_pos = SkyCoord(83.633, 22.014, unit="deg", frame='icrs')

We create a dictionary where we will store final results which we produce along the way.

In [ ]:
final_results= {}

We are creating a config object:

In [ ]:
config = AnalysisConfig()

Now we fill in the information in this config object. First we fill in the information that we are looking for observations within 2.5 degrees around our source position.

In [ ]:
config.observations.datastore = path
config.observations.obs_cone = {
    "frame": "icrs",
    "lon": source_pos.ra,
    "lat": source_pos.dec,
    "radius": 2.5 * u.deg,
}

The following parameters will be needed later, when we want to create sky maps.

In [ ]:
config.datasets.type = "3d"
config.datasets.geom.wcs.skydir = {
    "lon": source_pos.ra,
    "lat": source_pos.dec,
    "frame": "icrs",
} 
config.datasets.geom.wcs.fov = {"width": "5 deg", "height": "5 deg"}
config.datasets.geom.wcs.binsize = "0.02 deg"

# The FoV radius to use for cutouts
config.datasets.geom.selection.offset_max = 5 * u.deg

# We now fix the energy axis for the counts map - (the reconstructed energy binning)
config.datasets.geom.axes.energy.min = "0.5 TeV"
config.datasets.geom.axes.energy.max = "5 TeV"
config.datasets.geom.axes.energy.nbins = 10

# We need to extract the ring for each observation separately, hence, no stacking at this stage
config.datasets.stack = False

In [ ]:
print(config)

Now we create an analysis object for this configuration:

In [ ]:
analysis = Analysis(config)

In [ ]:
# for this specific case,w e do not need fine bins in true energy
analysis.config.datasets.geom.axes.energy_true = (
    analysis.config.datasets.geom.axes.energy
)

In [ ]:
# `First get the required observations
analysis.get_observations()

In [ ]:
analysis.get_datasets()

In [ ]:
analysis.observations.ids

## Event Lists
First we will have a look at the data as it is found in the files. This will give you an idea about the data and you will learn how to program yourself a very quick analysis.
### Single run and theta^2 plot

In [ ]:
runno = '23523'

In [ ]:
events_run = analysis.observations[runno].events

In [ ]:
print(events_run)

In [ ]:
events_run.table

In [ ]:
events_run.peek()

In [ ]:
events_run.plot_image()

In [ ]:
events_run.radec

In [ ]:
print(source_pos)

In [ ]:
theta2 = events_run.radec.separation(source_pos)**2

In [ ]:
print(theta2)

In [ ]:
ret = plt.hist(theta2.value, range = [0,0.1], bins = 50)

In [ ]:
ret

In [ ]:
n = ret[0]
x = ret[1]

In [ ]:
print(x)

In [ ]:
x[1:]<0.01

In [ ]:
n[ x[1:]<0.01 ]

In [ ]:
oncounts = n[x[1:]<0.01].sum()

In [ ]:
oncounts

find background region

In [ ]:
obs = analysis.observations[runno]

In [ ]:
print(obs)

In [ ]:
obs.pointing_radec

In [ ]:
separation = obs.pointing_radec.separation(source_pos)
print (separation)

In [ ]:
position_angle = obs.pointing_radec.position_angle(source_pos)
print (position_angle.to(u.deg))

In [ ]:
offpos = obs.pointing_radec.directional_offset_by( position_angle+180*u.deg, separation)

In [ ]:
offpos

In [ ]:
theta2_off = events_run.radec.separation(offpos)**2

In [ ]:
#plt.hist(theta2.value, range = [0,0.1], bins = 50)
ret_off = plt.hist(theta2_off.value, range = [0,0.1], bins = 50, alpha = 0.5)

In [ ]:
n_off = ret_off[0]
offcounts = n_off[x[1:]<0.01].sum()

In [ ]:
print(oncounts,offcounts)

In [ ]:
from gammapy.stats import WStatCountsStatistic
stat = WStatCountsStatistic(n_on=oncounts, n_off=offcounts, alpha=1.)
print(stat.excess,stat.significance)

#### Your playground
If you want to do it yourself, try the following.

Make a theta^2 plot for signal and background for run no. 23523. Calculate the number of on and off events for a theta^2 cut of 0.02. Calculate the significance.

In [ ]:
## your code here

### Combined event list of all runs
We have four observation runs. Let's see how we can combine them.

In [ ]:
analysis.observations.ids

In [ ]:
l = list(map(lambda x : analysis.observations[x].events, analysis.observations.ids))

In [ ]:
l

In [ ]:
events = EventList.stack(l)

In [ ]:
events.table

In [ ]:
len(events.table)

In [ ]:
events.peek()

With this event list you cannot make a theta-square plot with the simple steps shown above. Because every run has a different observation position, and by stacking all events you lost this information.

## Sky Maps
### Simple Counts Map

In [ ]:
map_crab = Map.create(binsz=0.01, width=(5, 5), skydir=source_pos, frame='icrs')

In [ ]:
map_crab.plot()

In [ ]:
events.radec

In [ ]:
map_crab.fill_by_coord(events.radec)

In [ ]:
map_crab.plot()

In [ ]:
smoothed = map_crab.smooth(width=0.05 * u.deg, kernel="gauss")

In [ ]:
smoothed.plot(stretch="log", add_cbar=True)

#### Your playground
Make a smoothed sky map. Make the map 3degx3deg large, and use a binning of 0.005deg. Smooth with a Gaussian kernel with 0.03deg. Add a colour bar and strech such that both the source and the background is clearly seen. You can also try different colour schemes, for example cmap='ocean_r'.

In [ ]:
## your code here

### Excess and Significance Maps
You have seen in the above example that there is a lot of background noise in the map. We want to subtract the background and obtain the excess events, and we want a map indicating the significance at each point of the sky. We will use the prebuild RingBackgroundMaker for this.

We start defining the geometry (i.e. the coordinate system and extension of the map) and energy axis for the map.

In [ ]:
geom = analysis.datasets[0].counts.geom
energy_axis = analysis.datasets[0].counts.geom.get_axis_by_name("energy")
geom_image = geom.to_image().to_cube([energy_axis.squash()])

Next we define an exclusion mask. This tells the RingBackgroundMaker which part of the sky must not be used for background estimation. We do not want the source itself in the background.

In [ ]:
regions = CircleSkyRegion(center=source_pos, radius=0.3 * u.deg)
exclusion_mask = Map.from_geom(geom_image)
exclusion_mask.data = geom_image.region_mask([regions], inside=False)
exclusion_mask.sum_over_axes().plot();

In [ ]:
ring_maker = RingBackgroundMaker(
    r_in="0.5 deg", width="0.3 deg", exclusion_mask=exclusion_mask
)

In [ ]:
#%%time
stacked_on_off = MapDatasetOnOff.create(geom=geom_image)
for dataset in analysis.datasets:
    dataset_image = (
        dataset.to_image()
    )  # Ring extracting makes sense only for 2D analysis
    dataset_on_off = ring_maker.run(dataset_image)
    stacked_on_off.stack(dataset_on_off)

In [ ]:
print(stacked_on_off)

In [ ]:
stacked_on_off.counts.sum_over_axes().plot(add_cbar = True, stretch = 'log')

In [ ]:
stacked_on_off.counts_off.sum_over_axes().plot(add_cbar = True)

In [ ]:
stacked_on_off.alpha.sum_over_axes().plot(add_cbar = True)

In [ ]:
excess_map = stacked_on_off.counts.sum_over_axes() - stacked_on_off.counts_off.sum_over_axes()*stacked_on_off.alpha.sum_over_axes()

In [ ]:
excess_map.plot(add_cbar = True)

In [ ]:
excess_map.smooth(width=0.05 * u.deg, kernel="gauss").plot(add_cbar = True, stretch='log')

The next cell shows how this can be done using gammapy internal functions. It also creates a significance map.

In [ ]:
# Using a convolution radius of 0.04 degrees
estimator = ExcessMapEstimator(0.04 * u.deg)
lima_maps = estimator.run(stacked_on_off, steps="ts")

full_significance_map = lima_maps["significance"]
full_excess_map = lima_maps["excess"]

significance_map = full_significance_map.get_image_by_idx((0,))
excess_map = full_excess_map.get_image_by_idx((0,))

# We can plot the excess and significance maps
plt.figure(figsize=(10, 10))
ax1 = plt.subplot(221, projection=significance_map.geom.wcs)
ax2 = plt.subplot(222, projection=excess_map.geom.wcs)

ax1.set_title("Significance map")
significance_map.plot(ax=ax1, add_cbar=True)

ax2.set_title("Excess map")
excess_map.plot(ax=ax2, add_cbar=True)

This looks good. Let's keep the maps for later.

In [ ]:
final_results['excess map'] = excess_map.copy()
final_results['significance map'] = significance_map.copy()

### your playground
Please try different convolution radii. You will see that smaller radii will lead to a more noisy image and larger radii will make the source appear bigger. This does not mean that the source is indeed bigger. You just smear out the emission over a larger area.

In [ ]:
## your code here

## Spectrum
Now we want to make an energy spectrum. We want to know the gamma-ray photon flux from the source at different energies.

In [ ]:
events_run.plot_energy()

In [ ]:
obs = analysis.observations[runno]

In [ ]:
obs.aeff.peek()

Let's define a target region.

In [ ]:
on_region_radius = Angle("0.11 deg")
on_region = CircleSkyRegion(center=source_pos, radius=on_region_radius)

First we have to check that our on region is sufficiently large to encompass all of the emission. We will plot the on region on our significance map created above.

In [ ]:
significance_map.plot(add_cbar = True)
on_region.to_pixel(significance_map.geom.wcs).plot(color = 'white')

Maybe we should make the radius a bit larger...

If there are other sources in the field of view they have to be masked. We do not see anything in the map, but there is an AGN nearby. We can exclude it just to be on the safe side. (And for illustration.)

In [ ]:
exclusion_region = CircleSkyRegion(
    center=SkyCoord(183.604, -8.708, unit="deg", frame="galactic"),
    radius=0.5 * u.deg,
)

exclusion_mask = Map.create(
    npix=(150, 150), binsz=0.05, skydir=source_pos, proj="TAN", frame="icrs"
)

mask = exclusion_mask.geom.region_mask([exclusion_region], inside=False)
exclusion_mask.data = mask
exclusion_mask.plot();

Let's define the energy binning. E_reco is the reconstructed energy of the photons, e_true is the true energy. Then we create an empty data set which will store the results in the end.

In [ ]:
e_reco = np.logspace(-1, np.log10(40), 40) * u.TeV
e_true = np.logspace(np.log10(0.05), 2, 200) * u.TeV
dataset_empty = SpectrumDataset.create(
    e_reco=e_reco, e_true=e_true, region=on_region
)

We need a maker to create the data set, another one to find the off-source regions, and a third one to select a good energy range.

In [ ]:
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False, selection=["counts", "aeff", "edisp"]
)
bkg_maker = ReflectedRegionsBackgroundMaker(exclusion_mask=exclusion_mask)
safe_mask_masker = SafeMaskMaker(methods=["aeff-max"], aeff_percent=10)

In [ ]:
datasets = Datasets()

for obs_id, observation in zip(analysis.observations.ids, analysis.observations):
    dataset = dataset_maker.run(
        dataset_empty.copy(name=str(obs_id)), observation
    )
    dataset_on_off = bkg_maker.run(dataset, observation)
    dataset_on_off = safe_mask_masker.run(dataset_on_off, observation)
    datasets.append(dataset_on_off)

In [ ]:
plt.figure(figsize=(8, 8))
_, ax, _ = exclusion_mask.plot()
on_region.to_pixel(ax.wcs).plot(ax=ax, edgecolor="k")
plot_spectrum_datasets_off_regions(ax=ax, datasets=datasets)

Important note: If we want to plot all this on top of the significance map then we have to make sure that the maps have the same size. We do so by creating a new map and filling it with the old one.

In [ ]:
sig2 = WcsNDMap(exclusion_mask.geom)

In [ ]:
sig2.coadd(significance_map)

In [ ]:
plt.figure(figsize=(8, 8))
_, ax, _ = exclusion_mask.plot()
on_region.to_pixel(ax.wcs).plot(ax=ax, edgecolor="k")
plot_spectrum_datasets_off_regions(ax=ax, datasets=datasets)
sig2.plot(ax=ax)

In [ ]:
info_table = datasets.info_table(cumulative=True)

In [ ]:
info_table

The lines do not represent the individual runs. They are the sum of the runs. Line 1 is the first run, line 2 the first two runs and so on. We can make a plot how the signal develops over time:

In [ ]:
plt.plot(
    info_table["livetime"].to("h"), info_table["excess"], marker="o", ls="none"
)
plt.xlabel("Livetime [h]")
plt.ylabel("Excess");

In [ ]:
plt.plot(
    info_table["livetime"].to("h"),
    info_table["significance"],
    marker="o",
    ls="none",
)
plt.xlabel("Livetime [h]")
plt.ylabel("Significance");

Let's take a look at the last column (index -1), which contains the sum of all runs. And we look only at a few rows in the table.

In [ ]:
info_table[-1]['n_on','n_off','alpha','excess', 'significance']

You see that the excess corresponds to n_on - n_off*alpha:

In [ ]:
635.0 - 1165.0*0.044459566473960876

Let's keep some of these values for later.

In [ ]:
final_results['excess'] = info_table[-1]['excess']
final_results['significance'] = info_table[-1]['significance'][0]

### Spectrum Fit
Now we want to describe the energy distribution of the gamma rays with a function. We will make a spectral fit. We will stack all the data of all runs into one data set and proceed with the fit. We could also fit the model to each run individually.

In [ ]:
dataset_stacked = Datasets(datasets).stack_reduce()

We start with a simple power law. Remember, the power law is
$$
f(E) = A \times \left( \frac{E}{E_0} \right) ^{-\Gamma}.
$$
The amplitude $A$ and the spectral index $\Gamma$ are free parameters in the fit. $E_0$ is the reference energy, which is not fitted. You can freely chose the value of $E_0$, but it is best to keep it within the energy range of the data.

In [ ]:
spectral_model = PowerLawSpectralModel(
    index=2, amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), reference=1 * u.TeV
)
model = SkyModel(spectral_model=spectral_model, name="crab")

In [ ]:
dataset_stacked.models = model
stacked_fit = Fit([dataset_stacked])
result_stacked = stacked_fit.run()

# make a copy to compare later
model_best_PL = model.copy()

In [ ]:
print(result_stacked)

Let's save the total stat for later.

In [ ]:
L_PL = result_stacked.total_stat

In [ ]:
plt.figure(figsize=(8, 6))
ax_spectrum, ax_residual = dataset_stacked.plot_fit()
#ax_spectrum.set_ylim(0.1, 100)

In [ ]:
model_best_PL.parameters.to_table()

In [ ]:
plot_kwargs = {
    "energy_range": [0.1, 30] * u.TeV,
    "energy_power": 2,
    "flux_unit": "erg-1 cm-2 s-1",
}

# plot stacked model
model_best_PL.spectral_model.plot(
    **plot_kwargs, label="Stacked analysis result"
)
model_best_PL.spectral_model.plot_error(**plot_kwargs)

# plot joint model
#model_best_joint.spectral_model.plot(
#    **plot_kwargs, label="Joint analysis result", ls="--"
#)
#model_best_joint.spectral_model.plot_error(**plot_kwargs)

create_crab_spectral_model("hess_pl").plot(
    **plot_kwargs, label="Crab reference"
)
plt.legend()

Let's do the spectrum again, this time we want to fit a power law with an exponential cut-off at high energies. This function is defined as
$$
f(E) = A \times \left( \frac{E}{E_0} \right)^{-\Gamma} \times \exp \left(-\frac{E}{E_c} \right).
$$
The last term can be written as
$$
\exp \left(-\frac{E}{E_c} \right) = \exp \left(-\lambda E \right)
$$
with $\lambda = 1/E_c$.

In [ ]:
spectral_model = ExpCutoffPowerLawSpectralModel(
    index=2.6, amplitude=2.7e-11 * u.Unit("cm-2 s-1 TeV-1"), reference=1 * u.TeV, lambda_ = 1./(10*u.TeV)
)
model = SkyModel(spectral_model=spectral_model, name="crab")

In [ ]:
dataset_stacked.models = model
stacked_fit = Fit([dataset_stacked])
result_stacked = stacked_fit.run()

# make a copy to compare later
model_best_expPL = model.copy()

In [ ]:
print(result_stacked)

In [ ]:
L_expPL = result_stacked.total_stat

In [ ]:
plt.figure(figsize=(8, 6))
ax_spectrum, ax_residual = dataset_stacked.plot_fit()

In [ ]:
model_best_expPL.parameters.to_table()

What is the cut-off energy?

In [ ]:
1/(model_best_expPL.parameters['lambda_'].quantity)

In [ ]:
plot_kwargs = {
    "energy_range": [0.1, 30] * u.TeV,
    "energy_power": 2,
    "flux_unit": "erg-1 cm-2 s-1",
}

# plot stacked model
model_best_expPL.spectral_model.plot(
    **plot_kwargs, label="Stacked analysis result"
)
model_best_expPL.spectral_model.plot_error(**plot_kwargs)

create_crab_spectral_model("hess_pl").plot(
    **plot_kwargs, label="Crab reference"
)
plt.legend()

Which model is better, with or without the cut-off? We will need to compare the likelyhoods of the fits.

In [ ]:
print(L_PL, L_expPL)

In [ ]:
TS = 2*(L_PL-L_expPL)
print(TS)

In [ ]:
P = scipy.stats.chi2.sf(TS,1)
print('probabilty: ',P)

In [ ]:
print('significant?', P < 2.7e-3)

The model with the cut-off is  significantly better. So we will use this model for our further analysis.

In [ ]:
model_best_stacked = model_best_expPL

In [ ]:
print(model_best_stacked)

In [ ]:
dataset_stacked.models = model_best_stacked

And we store the fit parameters in the final results.

In [ ]:
final_results['fit parameters'] = model_best_stacked.parameters.to_table()

final_results['model type'] = model_best_stacked.spectral_model.tag

### Your playground
You can try to fit yet another model. A LogParabolaSpectralModel could work as well. Do not forget to import the model. Check the documentation (https://docs.gammapy.org/0.17/api/gammapy.modeling.models.LogParabolaSpectralModel.html#gammapy.modeling.models.LogParabolaSpectralModel) for the parameters of this model.
You can compare the log-parabola model with the power law. But you cannot use Wilk's theorem to compare it to the exponential cut-off power law, as they are not nested models.

In [ ]:
## your code here

### Spectrum Points
Now we will generate spectral points for our best fit. The spectral points show the measured flux in each energy bin if the best-fit model is true.

In [ ]:
e_min, e_max = 0.7, 30
e_edges = np.logspace(np.log10(e_min), np.log10(e_max), 11) * u.TeV

In [ ]:
fpe = FluxPointsEstimator(e_edges=e_edges, source=model_best_stacked.name)
flux_points = fpe.run(datasets=dataset_stacked)

In [ ]:
flux_points.table_formatted

Some flux points are not significant. We want all points with TS < 4 be labelled as not significant and we want to use the upper limit instead.

In [ ]:
flux_points.table["is_ul"] = flux_points.table["ts"] < 4

We don't need all information of the flux point table. Let's reduce it a bit.

In [ ]:
final_fluxpoints = flux_points.table_formatted['e_ref', 'e_min', 'e_max', 'ref_dnde', 'dnde_err','counts','is_ul','dnde_ul']
final_fluxpoints

Let's keep this table for our final results.

In [ ]:
final_results['flux points'] = final_fluxpoints

Let's make a plot of our flux points.

In [ ]:
flux_points_dataset = FluxPointsDataset(
    data=flux_points, models=model_best_stacked
)

plt.figure(figsize=(8, 6))
flux_points_dataset.peek()

In [ ]:
flux_points.table_formatted['ref_e2dnde'].to(u.erg/(u.cm**2 * u.s))

In [ ]:
flux_points_dataset.write('Crab.dat', format = 'ipac')

## Your playground
You can make a spectrum with more or less data points.

In [ ]:
## your code here

## Summary

That's all. Let's see what we have and summarise.

In [ ]:
final_results.keys()

In [ ]:
print('We have detected an excess of {:4.1f} gamma rays with a statistical significance of {:3.1f} sigma.'.format(final_results['excess'], final_results['significance']))

In [ ]:
print('The spectrum is best described by a {:s}.'.format(final_results['model type']))

In [ ]:
print('The best fit parameters are:\n', final_results['fit parameters'])

In [ ]:
print('The spectral data points are:\n', final_results['flux points'])

In [ ]:
final_results['excess map'].plot(stretch='log', add_cbar = 'true')

In [ ]:
final_results['significance map'].plot(add_cbar = 'true')